# Notebook 3 — Semantic Search on the ICON Model Code Base

In this notebook we build a **practical, reusable semantic search system**
for the open-source ICON weather and climate model.

We will:
- clone the ICON Git repository,
- extract and chunk relevant text and code files,
- build a FAISS vector index,
- and perform meaningful semantic queries.

This notebook focuses on **clean structure, reproducibility, and usability**.
Generated data (document copies, embeddings, FAISS indices) are **excluded from Git**.

In [2]:
# ------------------------------------------------------------
# Project structure and paths
# ------------------------------------------------------------
from pathlib import Path

# Root of this notebook (assumed to live inside the course repo)
PROJECT_ROOT = Path(".").resolve()

# Where external repositories are cloned
DOCUMENTS_DIR = PROJECT_ROOT / "documents"

# Where FAISS indices and metadata are stored
INDEX_DIR = PROJECT_ROOT / "vector_db"

# Create directories if they do not exist
DOCUMENTS_DIR.mkdir(exist_ok=True)
INDEX_DIR.mkdir(exist_ok=True)

print("PROJECT_ROOT :", PROJECT_ROOT)
print("DOCUMENTS_DIR:", DOCUMENTS_DIR)
print("INDEX_DIR    :", INDEX_DIR)


PROJECT_ROOT : /Users/rpotthas/all/e-ai_ml2/course/code/code07
DOCUMENTS_DIR: /Users/rpotthas/all/e-ai_ml2/course/code/code07/documents
INDEX_DIR    : /Users/rpotthas/all/e-ai_ml2/course/code/code07/vector_db


In [3]:
# ------------------------------------------------------------
# Ensure generated folders are ignored by Git
# ------------------------------------------------------------
gitignore_path = PROJECT_ROOT / ".gitignore"

ignore_entries = [
    "documents/",
    "vector_db/",
    "*.index",
    "*.faiss",
    "__pycache__/",
    ".ipynb_checkpoints/"
]

existing = set()
if gitignore_path.exists():
    existing = set(gitignore_path.read_text().splitlines())

with gitignore_path.open("a", encoding="utf-8") as f:
    for entry in ignore_entries:
        if entry not in existing:
            f.write(entry + "\n")
            print("Added to .gitignore:", entry)

Added to .gitignore: documents/
Added to .gitignore: vector_db/
Added to .gitignore: *.index
Added to .gitignore: *.faiss
Added to .gitignore: __pycache__/
Added to .gitignore: .ipynb_checkpoints/


In [5]:
# ------------------------------------------------------------
# Proxy configuration (DWD network)
# ------------------------------------------------------------
import os

PROXY = "http://ofsquid.dwd.de:8080"

os.environ["HTTP_PROXY"]  = PROXY
os.environ["HTTPS_PROXY"] = PROXY
os.environ["http_proxy"]  = PROXY
os.environ["https_proxy"] = PROXY

# Do not proxy local connections
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

print("Proxy configured for HTTP/HTTPS:")
print(" ", PROXY)

Proxy configured for HTTP/HTTPS:
  http://ofsquid.dwd.de:8080


In [6]:
# ------------------------------------------------------------
# Clone ICON model repository (shallow clone)
# ------------------------------------------------------------
import subprocess

ICON_REPO_URL = "https://gitlab.dkrz.de/icon/icon-model.git"
ICON_DIR = DOCUMENTS_DIR / "icon-model"

if not ICON_DIR.exists():
    subprocess.run(
        ["git", "clone", "--depth", "1", ICON_REPO_URL, str(ICON_DIR)],
        check=True
    )
    print("ICON repository cloned.")
else:
    print("ICON repository already exists:", ICON_DIR)

Klone nach '/Users/rpotthas/all/e-ai_ml2/course/code/code07/documents/icon-model'...


ICON repository cloned.


In [8]:
# ------------------------------------------------------------
# Sanity check: list top-level ICON repository content
# ------------------------------------------------------------
for p in sorted(ICON_DIR.iterdir()):
    print(p.name)


.clang-format
.cmake-format.py
.fprettify.rc
.git
.git-blame-ignore-revs
.gitattributes
.gitignore
.gitlab
.gitlab-ci.yml
.gitmodules
.pre-commit-config.yaml
AUTHORS.TXT
CONTRIBUTING.md
LICENSES
Makefile.in
README.md
RELEASE_NOTES.md
REUSE.toml
collect.extra-libs.in
config
config.h.in
configure
configure.ac
data
depgen.c.config.in
depgen.f90.config.in
deplist.config.in
doc
etc
externals
icon.mk.in
inlib.mk.in
m4
make_runscripts
ragnarok
run
schedulers
scripts
src
support
test
utils
vertical_coord_tables


In [9]:
# ------------------------------------------------------------
# File selection rules for ICON repository
# ------------------------------------------------------------
from pathlib import Path

# Directories we explicitly skip
SKIP_DIRS = {
    ".git",
    ".gitlab",
    "externals",      # external dependencies
    "data",           # often large, non-text
    "LICENSES"
}

# File extensions we consider searchable text/code
TEXT_EXTENSIONS = {
    ".f90", ".F90", ".f", ".F",
    ".c", ".h", ".cpp", ".hpp",
    ".py", ".sh",
    ".txt", ".md", ".rst", ".org",
    ".cfg", ".ini", ".nml",
    ".yaml", ".yml", ".toml",
    ".mk"
}

# Some important files have no extension
ALLOW_EXTENSIONLESS = {
    "configure",
    "make_runscripts",
    "README",
    "NOTICE"
}

# Safety limit: skip very large files
MAX_FILE_SIZE = 2_000_000  # 2 MB


def is_searchable_file(path: Path) -> bool:
    """Decide whether a file should be indexed."""
    if not path.is_file():
        return False

    # Skip unwanted directories
    if any(part in SKIP_DIRS for part in path.parts):
        return False

    # Skip large files
    if path.stat().st_size > MAX_FILE_SIZE:
        return False

    # Extension-based check
    if path.suffix in TEXT_EXTENSIONS:
        return True

    # Extensionless but known-important files
    if path.suffix == "" and path.name in ALLOW_EXTENSIONLESS:
        return True

    return False


In [12]:
# ------------------------------------------------------------
# Preview which files will be indexed
# ------------------------------------------------------------
searchable_files = []

for p in ICON_DIR.rglob("*"):
    if is_searchable_file(p):
        searchable_files.append(p)

print(f"Number of searchable files: {len(searchable_files)}\n")

for p in searchable_files[:30]:
    print(p.relative_to(ICON_DIR))

Number of searchable files: 1211

REUSE.toml
make_runscripts
configure
.pre-commit-config.yaml
README.md
CONTRIBUTING.md
RELEASE_NOTES.md
.gitlab-ci.yml
.cmake-format.py
etc/o3_cmip_provider.py
etc/aero_provider.py
utils/test_driver.py
utils/icon_sorted_deps.sh
utils/pvcs.py
utils/move_to_prefix.sh
support/mo_util_uuid.f90
support/index_list.h
support/util_multifile_restart.c
support/mo_util_uuid_types.f90
support/util_uuid.h
support/util_uuid.c
vertical_coord_tables/README
ragnarok/CMakeLists.txt
ragnarok/README.md
run/create-hostfile.py
run/ana_varnames_map_file.txt
run/README
run/run_wrapper.sh
run/postFunx.sh
test/unit-tests/test_start_mpi.mpi.f90


## Chunking strategy

We do **not** embed whole files.

Reasons:
- Files are often too long for meaningful embeddings
- Queries usually target *local* functionality
- Chunking improves retrieval precision

We therefore split files into **overlapping text chunks**:
- small enough to be specific
- large enough to preserve context

Each chunk will later be stored as one vector in the FAISS index.

In [13]:
# ------------------------------------------------------------
# Utilities: safe text reading and normalization
# ------------------------------------------------------------
import re

def read_text_file(path: Path) -> str:
    """
    Read a text file robustly.
    Try UTF-8 first, fall back to latin-1 if needed.
    """
    try:
        return path.read_text(encoding="utf-8")
    except UnicodeDecodeError:
        return path.read_text(encoding="latin-1", errors="replace")


def normalize_text(text: str) -> str:
    """
    Normalize whitespace and line endings.
    Keeps line structure (important for code).
    """
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    # Collapse excessive empty lines
    text = re.sub(r"\n{4,}", "\n\n\n", text)
    return text

In [14]:
# ------------------------------------------------------------
# Chunking function (line-based, overlapping)
# ------------------------------------------------------------
from typing import List, Dict

def chunk_text_by_lines(
    text: str,
    target_chars: int = 1800,
    overlap_chars: int = 300
) -> List[Dict]:
    """
    Split text into overlapping chunks based on line structure.

    Parameters
    ----------
    text : str
        Full text of a file.
    target_chars : int
        Approximate size of each chunk (in characters).
    overlap_chars : int
        Overlap between consecutive chunks (in characters).

    Returns
    -------
    List of dicts with:
        - 'text'
        - 'line_start'
        - 'line_end'
    """
    lines = text.splitlines()
    chunks = []

    buffer = []
    buffer_len = 0
    start_line = 1

    for i, line in enumerate(lines):
        buffer.append(line)
        buffer_len += len(line) + 1

        if buffer_len >= target_chars:
            chunk_text = "\n".join(buffer).strip()
            end_line = i + 1

            chunks.append({
                "text": chunk_text,
                "line_start": start_line,
                "line_end": end_line,
            })

            # Prepare overlap
            overlap_buffer = []
            overlap_len = 0
            for prev_line in reversed(buffer):
                overlap_buffer.insert(0, prev_line)
                overlap_len += len(prev_line) + 1
                if overlap_len >= overlap_chars:
                    break

            buffer = overlap_buffer
            buffer_len = overlap_len
            start_line = end_line - len(buffer) + 1

    # Remaining text
    if buffer:
        chunks.append({
            "text": "\n".join(buffer).strip(),
            "line_start": start_line,
            "line_end": len(lines),
        })

    return chunks

In [16]:
# ------------------------------------------------------------
# Build chunks from ICON repository
# ------------------------------------------------------------
chunks = []

for path in searchable_files:
    raw_text = read_text_file(path)
    text = normalize_text(raw_text)

    if len(text.strip()) < 100:
        continue  # skip trivial files

    file_chunks = chunk_text_by_lines(text)

    for i, ch in enumerate(file_chunks):
        chunks.append({
            "file": str(path.relative_to(ICON_DIR)),
            "chunk_id": i,
            "line_start": ch["line_start"],
            "line_end": ch["line_end"],
            "text": ch["text"],
        })

print(f"Total chunks created: {len(chunks)}")

Total chunks created: 26883


In [17]:
# ------------------------------------------------------------
# Inspect a few chunks manually
# ------------------------------------------------------------
for c in chunks[:3]:
    print("=" * 80)
    print(f"File: {c['file']}")
    print(f"Lines: {c['line_start']}–{c['line_end']}")
    print("-" * 80)
    print(c["text"][:800])

File: REUSE.toml
Lines: 1–64
--------------------------------------------------------------------------------
# ICON
#
# ---------------------------------------------------------------
# Copyright (C) 2004-2025, DWD, MPI-M, DKRZ, KIT, ETH, MeteoSwiss
# Contact information: icon-model.org
# See AUTHORS.TXT for a list of authors
# See LICENSES/ for license information
# SPDX-License-Identifier: CC0-1.0
# ---------------------------------------------------------------

version = 1

[[annotations]]
# Documentation and its source files:
path = [
  ".gitlab/ci/**/README.md",
  "AUTHORS.TXT",
  "CONTRIBUTING.md",
  "README.md",
  "RELEASE_NOTES.md",
  "config/**/*.readme",
  "config/**/README.md",
  "data/README",
  "doc/**/*.eps",
  "doc/**/*.ico",
  "doc/**/*.jpg",
  "doc/**/*.md",
  "doc/**/*.pdf",
  "doc/**/*.png",
  "doc/**/*.svg",
  "ragnarok/README.md",
  "run/**/README*",
  "schedulers/ecmwf/doc/
File: REUSE.toml
Lines: 58–108
----------------------------------------------------------

In [18]:
# ------------------------------------------------------------
# Load sentence embedding model
# ------------------------------------------------------------
from sentence_transformers import SentenceTransformer

# Compact, fast, good semantic quality
MODEL_NAME = "all-MiniLM-L6-v2"

model = SentenceTransformer(MODEL_NAME)

print("Loaded embedding model:", MODEL_NAME)
print("Embedding dimension:", model.get_sentence_embedding_dimension())


Loaded embedding model: all-MiniLM-L6-v2
Embedding dimension: 384


In [20]:
# ------------------------------------------------------------
# Prepare chunk texts for embedding
# ------------------------------------------------------------
texts = [c["text"] for c in chunks]

print("Number of chunks:", len(texts))
print("Example text length:", len(texts[0]))

Number of chunks: 26883
Example text length: 1846


In [21]:
# ------------------------------------------------------------
# Compute embeddings
# ------------------------------------------------------------
import numpy as np

embeddings = model.encode(
    texts,
    convert_to_numpy=True,
    show_progress_bar=True,
    batch_size=32
)

# Ensure FAISS-compatible dtype
embeddings = embeddings.astype(np.float32)

print("Embedding array shape:", embeddings.shape)

Batches:   0%|          | 0/841 [00:00<?, ?it/s]

Embedding array shape: (26883, 384)


In [22]:
# ------------------------------------------------------------
# Sanity checks
# ------------------------------------------------------------
assert embeddings.ndim == 2
assert embeddings.shape[0] == len(chunks)
assert embeddings.shape[1] == 384

# Norm statistics (useful for cosine similarity)
norms = np.linalg.norm(embeddings, axis=1)
print("Embedding norms:")
print("  min:", norms.min())
print("  max:", norms.max())
print("  mean:", norms.mean())

Embedding norms:
  min: 0.9999999
  max: 1.0000001
  mean: 1.0


In [23]:
# ------------------------------------------------------------
# Inspect one embedded chunk
# ------------------------------------------------------------
i = 0

print("File:", chunks[i]["file"])
print("Lines:", chunks[i]["line_start"], "-", chunks[i]["line_end"])
print("-" * 80)
print(chunks[i]["text"][:600])
print("-" * 80)
print("Embedding (first 10 components):")
print(embeddings[i][:10])

File: REUSE.toml
Lines: 1 - 64
--------------------------------------------------------------------------------
# ICON
#
# ---------------------------------------------------------------
# Copyright (C) 2004-2025, DWD, MPI-M, DKRZ, KIT, ETH, MeteoSwiss
# Contact information: icon-model.org
# See AUTHORS.TXT for a list of authors
# See LICENSES/ for license information
# SPDX-License-Identifier: CC0-1.0
# ---------------------------------------------------------------

version = 1

[[annotations]]
# Documentation and its source files:
path = [
  ".gitlab/ci/**/README.md",
  "AUTHORS.TXT",
  "CONTRIBUTING.md",
  "README.md",
  "RELEASE_NOTES.md",
  "config/**/*.readme",
  "config/**/README.md",
  "data/RE
--------------------------------------------------------------------------------
Embedding (first 10 components):
[-0.11404255  0.09105776 -0.05297947 -0.0285041   0.02929485 -0.03326541
  0.03339936 -0.00376081  0.02134291 -0.01417605]


## Why FAISS?

Brute-force similarity search compares a query vector against **all** stored vectors:
- Cost: 𝒪(N · d) per query
- Too slow for large libraries

FAISS (Facebook AI Similarity Search) provides:
- Fast nearest-neighbor search
- Optimized C++ backend
- Exact or approximate algorithms
- CPU and GPU support

We start with the **simplest exact index**.


In [24]:
# ------------------------------------------------------------
# Build FAISS index (exact, cosine similarity)
# ------------------------------------------------------------
import faiss

d = embeddings.shape[1]   # embedding dimension
index = faiss.IndexFlatIP(d)  # inner product = cosine similarity (unit vectors)

index.add(embeddings)

print("FAISS index built")
print("Number of vectors:", index.ntotal)
print("Dimension:", d)

FAISS index built
Number of vectors: 26883
Dimension: 384


In [25]:
# ------------------------------------------------------------
# FAISS query function
# ------------------------------------------------------------
def search_faiss(query: str, top_k: int = 5):
    """
    Search the FAISS index for the most similar chunks.
    Returns a list of (score, metadata).
    """
    query_vec = model.encode(
        query,
        convert_to_numpy=True
    ).astype("float32").reshape(1, -1)

    scores, indices = index.search(query_vec, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        meta = chunks[idx]
        results.append((score, meta))

    return results

In [26]:
# ------------------------------------------------------------
# Example query
# ------------------------------------------------------------
query = "How is cloud microphysics treated in ICON?"

results = search_faiss(query, top_k=3)

for rank, (score, r) in enumerate(results, start=1):
    print("=" * 80)
    print(f"Rank {rank} | similarity = {score:.3f}")
    print("File:", r["file"])
    print("Lines:", r["line_start"], "-", r["line_end"])
    print("-" * 80)
    print(r["text"][:800])

Rank 1 | similarity = 0.631
File: doc/www/atmosphere/sbm/sbm_overview.md
Lines: 26 - 31
--------------------------------------------------------------------------------
ICON uses total water and ice contents (qc, qr, qi, qs, qg) in many places in the code. When using SBM, we keep advecting these total water contents, but overwrite them by the integrals over the corresponding size distributions, right after the call to the microphysics. The vertical turbulent diffusion "turbdiff" also uses cloud water, ice and snow (qc, qi, qs). For consistency, the corresponding SBM bins (tracers) were added to the vertical turbulent diffusion as well. Convection scheme might also detrain qc, qi, qr, qs. For mass conservation, we "evaporate" this water, transfering it to water wapor qv, and subtract the corresponding latent heat release. In appropreate conditions, this water vapor may condensate to hydrometeors as part of SBM. Total water contents are used in ICON initial
Rank 2 | similarity = 0.622
Fi

## Interpreting FAISS scores

Because vectors are normalized:
- Score ≈ 1.0 → very similar meaning
- Score ≈ 0.7 → related topic
- Score < 0.4 → weak relation

Exact values depend on:
- chunk size
- wording
- abstraction level

What matters is **relative ranking**, not absolute numbers.

## RAG step: Retrieve → Generate

RAG combines:
1. **Retrieval**: find the most relevant ICON chunks via FAISS
2. **Generation**: ask an LLM to answer using *only* the retrieved context

Key goals:
- answers grounded in the repository
- traceable references (file + line ranges)
- limited context size (avoid overflowing the model context)

In [28]:
# ------------------------------------------------------------
# Build a compact context from retrieved chunks
# ------------------------------------------------------------
def build_context(query: str, top_k: int = 8, max_chars: int = 12000):
    """
    Retrieve top_k chunks and build a context string that fits into max_chars.
    Returns (context_text, references_list).
    """
    results = search_faiss(query, top_k=top_k)

    context_parts = []
    refs = []
    total = 0

    for score, r in results:
        header = f"[{r['file']} | lines {r['line_start']}-{r['line_end']} | sim={score:.3f}]"
        body = r["text"].strip()

        block = header + "\n" + body
        block_len = len(block) + 2

        if total + block_len > max_chars:
            break

        context_parts.append(block)
        refs.append((r["file"], r["line_start"], r["line_end"], float(score)))
        total += block_len

    context_text = "\n\n---\n\n".join(context_parts)
    return context_text, refs


In [31]:
# ------------------------------------------------------------
# Load API keys from .env file
# ------------------------------------------------------------
from dotenv import load_dotenv
import os

# Load variables from .env in the current directory (or parent dirs)
load_dotenv()

# Sanity check (do NOT print the key itself)
if "OPENAI_API_KEY" not in os.environ:
    raise RuntimeError("OPENAI_API_KEY not found in environment")

print("OPENAI_API_KEY loaded.")

# ------------------------------------------------------------
# OpenAI backend (requires OPENAI_API_KEY in environment)
# ------------------------------------------------------------
import os

USE_OPENAI = True  # set False if you want local-only

if USE_OPENAI:
    from openai import OpenAI
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    print("OpenAI client initialized.")
else:
    print("OpenAI disabled.")

OPENAI_API_KEY loaded.
OpenAI client initialized.


In [37]:
# ------------------------------------------------------------
# RAG: Retrieval + OpenAI generation
# Streaming output with live Markdown display
# ------------------------------------------------------------
from IPython.display import display, Markdown, clear_output

def rag_answer_openai_streaming(
    query: str,
    top_k: int = 10,
    max_context_chars: int = 12000,
    model: str = "gpt-4o-mini",
):
    """
    Retrieve relevant ICON chunks via FAISS and generate
    a grounded Markdown answer using OpenAI with streaming output.
    """

    # --- Retrieval ------------------------------------------------------------
    results = search_faiss(query, top_k=top_k)

    context_parts = []
    total_chars = 0

    for score, r in results:
        header = (
            f"[{r['file']} | lines {r['line_start']}-{r['line_end']} "
            f"| similarity={score:.3f}]"
        )
        body = r["text"].strip()

        block = header + "\n" + body
        block_len = len(block) + 2

        if total_chars + block_len > max_context_chars:
            break

        context_parts.append(block)
        total_chars += block_len

    context = "\n\n---\n\n".join(context_parts)

    # --- Prompt ---------------------------------------------------------------
    system_prompt = (
        "You are a technical assistant for the ICON weather model code base. "
        "Answer strictly based on the provided context. "
        "If the answer is not contained in the context, say so explicitly. "
        "Write the answer in clear Markdown. "
        "Include a final section titled 'Sources' listing the file names "
        "and line ranges you used."
    )

    user_prompt = (
        f"## Question\n{query}\n\n"
        f"## Context (ICON repository excerpts)\n{context}\n\n"
        "## Instructions\n"
        "- Give a concise, technical answer\n"
        "- Use bullet points where appropriate\n"
        "- Do not introduce information not present in the context\n"
    )

    # --- Streaming OpenAI call ------------------------------------------------
    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
        stream=True,
    )

    accumulated_text = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        delta = chunk.choices[0].delta

        if delta and delta.content:
            accumulated_text += delta.content
            display_handle.update(Markdown(accumulated_text))

    return accumulated_text


In [38]:
query = "Where is the turbulence scheme implemented in ICON?"
answer_md = rag_answer_openai_streaming(query, top_k=12)

The turbulence scheme in the ICON model is implemented in several modules, primarily located in the following files:

- **Turbulence Utilities**:
  - **File**: `src/atm_phy_schemes/turb_utilities.f90`
  - **Key Routines**:
    - `turb_setup`: Setting up the turbulence model.
    - `init_basic_atmo_turb`: Initialization of basic properties of atmospheric turbulence.
    - `solve_turb_budgets`: Solution of prognostic TKE-equation and related equations.

- **Turbulent Mixing Interface**:
  - **File**: `src/atm_phy_aes/mo_interface_aes_tmx.f90`
  - **Functionality**: Calls the turbulent mixing scheme and surface schemes.

- **Vertical Diffusion Configuration**:
  - **File**: `src/configure_model/mo_turb_vdiff_config.f90`
  - **Description**: Configuration structure for the VDIFF turbulence scheme.

- **Vertical Diffusion Parameters**:
  - **File**: `src/atm_phy_schemes/mo_turb_vdiff_params.f90`
  - **Parameters**: Defines parameters for the VDIFF turbulence scheme.

- **Smagorinsky Turbulent Mixing**:
  - **File**: `src/atm_phy_aes/tmx/mo_tmx_smagorinsky.f90`
  - **Functionality**: Contains classes and functions for the Smagorinsky turbulent mixing scheme.

- **Numerics for Turbulent Mixing**:
  - **File**: `src/atm_phy_aes/tmx/mo_tmx_numerics.f90`
  - **Functionality**: Implements numerical methods for turbulent mixing processes.

These modules collectively handle various aspects of turbulence modeling within the ICON framework.

### Sources
- `src/atm_phy_schemes/turb_utilities.f90 | lines 1-35`
- `src/atm_phy_aes/mo_interface_aes_tmx.f90 | lines 1-48`
- `src/configure_model/mo_turb_vdiff_config.f90 | lines 1-39`
- `src/atm_phy_schemes/mo_turb_vdiff_params.f90 | lines 1-41`
- `src/atm_phy_aes/tmx/mo_tmx_smagorinsky.f90 | lines 1-58`
- `src/atm_phy_aes/tmx/mo_tmx_numerics.f90 | lines 1-53`